<a href="https://colab.research.google.com/github/suinkangme/comp333_project/blob/main/prediction_without_feature_scaling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **IMPORT THE LIBRARIES**

In [4]:
import os
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

## LOAD THE DATASET AFTER CLEANING & TRANSOFRMATION

In [5]:
# import the MyDrive library from colab
from google.colab import drive
drive.mount('/content/drive')

# the directory owns this notebook and dataset
os.chdir('/content/drive/MyDrive/comp333/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# load the data without scaling(transform)
dataset = pd.read_csv('no_scaling_dataset.csv')
print(dataset.columns)

Index(['Year', 'Atheists rate', 'Child mortality rate %',
       'Consumer price index', 'female labor force participation rate', 'GDP',
       'GDP per capita', 'Period life expectancy', 'Death rate', 'Population',
       ...
       'Entity_United States Virgin Islands', 'Entity_Uruguay',
       'Entity_Uzbekistan', 'Entity_Vanuatu', 'Entity_Venezuela',
       'Entity_Vietnam', 'Entity_Yemen', 'Entity_Zambia', 'Entity_Zimbabwe',
       'Birth rate(per 1000)'],
      dtype='object', length=230)


In [7]:
dataset.head()

,Year,Atheists rate,Child mortality rate %,Consumer price index,female labor force participation rate,GDP,GDP per capita,Period life expectancy,Death rate,Population,...,Entity_United States Virgin Islands,Entity_Uruguay,Entity_Uzbekistan,Entity_Vanuatu,Entity_Venezuela,Entity_Vietnam,Entity_Yemen,Entity_Zambia,Entity_Zimbabwe,Birth rate(per 1000)
0,1990.0,0.0,2.23419,49.299595,44.118387,1.577580e+09,30823.482,73.0760,918.566286,65734.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.662
1,2016.0,0.0,1.61821,102.438490,51.365839,3.025850e+09,37043.700,75.6168,721.915429,104890.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.300
2,1997.0,0.0,2.16597,66.445970,46.629871,2.325843e+09,36479.940,73.4524,849.212286,81872.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.388
3,2017.0,0.0,1.59065,101.385140,51.383419,3.191738e+09,38865.188,75.9030,721.057429,105454.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.530
4,2000.0,0.0,2.13121,72.031456,47.044387,2.584458e+09,37241.030,73.5690,819.967143,89117.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.427


# **BIRTH RATE PREDICTION BY COUNTRY**

In [8]:
X = dataset.iloc[:, :-1].values # input features
y = dataset.iloc[:, -1].values # target label : 'Birth rate(per 1000)'

In [9]:
# split the dataset into the train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

### Model 1 - Linear Regression

In [10]:
# train the Linear Regression model
from sklearn.linear_model import LinearRegression
model_1 = LinearRegression()
model_1.fit(X_train, y_train)

LinearRegression()

In [11]:
# model 1 evaluation
y_pred_1 = model_1.predict(X_test)
mse_1 = mean_squared_error(y_test, y_pred_1)

print(f'MSE from Model 1: {mse_1:.4f}')

MSE from Model 1: 4.6253


### Model 2 - SVM Regression

In [12]:
from sklearn.svm import SVR
model_2 = SVR(kernel = 'rbf')
model_2.fit(X_train, y_train)

SVR()

In [13]:
# model 2 evaluation
y_pred_2 = model_2.predict(X_test)
mse_2 = mean_squared_error(y_test, y_pred_2)

print(f'MSE from Model 2: {mse_2:.4f}')

MSE from Model 2: 111.9755


### Model 3 - MLP(Multi Layer Perceptron)

In [14]:
# dataloader
train_dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
test_dataset = TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))

batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [15]:
# define model 3
class Model(nn.Module):
    def __init__(self, input_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, 1)

    def forward(self, x):
        return self.fc(x)

In [16]:
input_size = X_train.shape[1]
model_3 = Model(input_size)

In [17]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model_3.parameters(), lr=0.001)

In [18]:
# training
num_epochs = 10

for epoch in range(num_epochs):
    model_3.train()
    total_loss = 0.0

    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model_3(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {total_loss / len(train_loader):.4f}')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/10, Train Loss: 922669265881768394752.0000
Epoch 2/10, Train Loss: 921529237396415971328.0000
Epoch 3/10, Train Loss: 920535141729425686528.0000
Epoch 4/10, Train Loss: 919751352341485453312.0000
Epoch 5/10, Train Loss: 918080923637493399552.0000
Epoch 6/10, Train Loss: 916848156384390807552.0000
Epoch 7/10, Train Loss: 915715332502828220416.0000
Epoch 8/10, Train Loss: 914768331459516170240.0000
Epoch 9/10, Train Loss: 913426783464252375040.0000
Epoch 10/10, Train Loss: 912218686591705284608.0000


In [19]:
# testing
model_3.eval()
with torch.no_grad():
    y_pred = []
    for inputs, targets in test_loader:
        outputs = model_3(inputs)
        y_pred.append(outputs.numpy())
y_pred = np.concatenate(y_pred)

mse_3 = mean_squared_error(y_test, y_pred)
print(f'MSE from Model 3: {mse_3:.4f}')

MSE from Model 3: 975997493775162540032.0000


In [20]:
print(f'MSE from Model 1(Linear Regression): {mse_1:.4f}')
print(f'MSE from Model 2(SVM Regression): {mse_2:.4f}')
print(f'MSE from Model 3(MLP): {mse_3:.4f}')

MSE from Model 1(Linear Regression): 4.6253
MSE from Model 2(SVM Regression): 111.9755
MSE from Model 3(MLP): 975997493775162540032.0000


=> Best Model : **Model 1(Linear Regression)**